In [1]:
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog
import numpy as np
import tensorflow as tf
import colorsys
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
def load_img():
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="Desktop/Skripsi/Data Uji/Data Testing/", title="Pilih Gambar",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png")))
    basewidth = 300 # Processing image for dysplaying
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper()).pack()
    panel_image = tk.Label(frame, image=img).pack()

In [3]:
def classify():
    start_time = time.time()
    image = Image.open(image_data)
    image = image.resize((200, 200), Image.ANTIALIAS)
    pixels = image.load()

    hls_array = np.empty(shape=(image.height, image.width, 3), dtype=float)
    hue = []
    light = []
    sat = []

    for row in range(0, image.height):
        for column in range(0, image.width):
            rgb = pixels[column, row]
            hls = colorsys.rgb_to_hls(rgb[0]/255, rgb[1]/255, rgb[2]/255)
            hue.append(np.round(hls[0]*359))
            light.append(np.round(hls[1]*100))
            sat.append(np.round(hls[2]*100))
            
    new_dict = {new_list: [] for new_list in range(361)}
    unique, counts = np.unique(hue, return_counts=True)
    dictry = dict(zip(unique, counts))
    merge = {**new_dict, **dictry}
    vall = list(merge.values())
    for i in range(len(vall)):
        if vall[i].__class__ is list :
            vall[i] = 0
    
    barang = np.array(vall)
    df = pd.DataFrame(barang)
    df = df.transpose()
    predbarang = model.predict(df)
    
    hasil = np.argmax(predbarang)
    confidence = np.max(predbarang)
    
    if hasil == 0 :
        identify = "Perhiasan Belum Pudar"
    else :
        identify = "Perhiasan Sudah Pudar"
    confidence = np.max(predbarang)
    times = time.time() - start_time
    table = tk.Label(frame, text="Hasil Identifikasi :").pack()
    result = tk.Label(frame,text= identify).pack()
    conf = tk.Label(frame, text="\nConfidence :").pack()
    confidence = tk.Label(frame,text= confidence).pack()
    tml = tk.Label(frame, text="\nWaktu Eksekusi Program :").pack()
    acttime = tk.Label(frame,text= str(("%s detik" % (times)))).pack()

In [4]:
root = tk.Tk()
root.title('Identifikasi Kepudaran Warna Perhiasan Lapis Emas')
root.iconbitmap('class.ico')
root.resizable(False, False)
tit = tk.Label(root, text="Created by Muhammad Harsye Ibra 54418669", padx=25, pady=6, font=("", 12)).pack()
canvas = tk.Canvas(root, height=500, width=500, bg='grey')
canvas.pack()
frame = tk.Frame(root, bg='white')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)
chose_image = tk.Button(root, text='Pilih Gambar',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=load_img)
chose_image.pack(side=tk.LEFT)
class_image = tk.Button(root, text='Identifikasi',
                        padx=35, pady=10,
                        fg="black", bg="yellow", command=classify)
class_image.pack(side=tk.RIGHT)
model = tf.keras.models.load_model('saved_model/my_model90%')
root.mainloop()